In [ ]:
from itertools import combinations



generate 35 permutations of 4 groups - 8 choose 4, but will some repeats where incl = x, excl = y vs incl = y, excl = x

In [ ]:
def generate_consistent_combinations(strings_list):
    all_combos = list(combinations(strings_list, 4))
    unique_pairs = []
    seen = set()
    
    for combo in all_combos:
        # Find the complement (the 4 items not in this combination)
        complement = tuple(c for c in strings_list if c not in combo)
        
        # order is alphabetic so we end up with duplicate (s1, s2), (s1, s2) instead of (s1, s2), (s2, s1)
        if combo < complement:
            pair = (combo, complement)
        else:
            pair = (complement, combo)

        pair_str = str(pair)  
        if pair_str not in seen:
            unique_pairs.append(pair)
            seen.add(pair_str)
    
    return unique_pairs

In [ ]:
best_concepts = [
    "A.Fayed", 
    "R.Wallace", 
    "T.Lennox", 
    "N.Yassir", 
    "K.Hayes", 
    "M.OBrian", 
    "J.Bauer", 
    "C.Manning"
]
combinations = generate_consistent_combinations(best_concepts)

Next: train decoders on 4x4 concepts

don't want overlap of each concept with the 4 opposing concepts - overlap with 3 other ones is fine

